In [64]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from mlxtend.preprocessing import TransactionEncoder
%pip install mlxtend==0.23.1

Note: you may need to restart the kernel to use updated packages.


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

you can refer to this article to find out about apriori and association rule:
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here:
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [65]:
# load the data set ans show the first five transaction
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv')
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [66]:
unique_items = set(np.ravel(df))
for i in df:
  unique_items.update(df[i].unique())
print(unique_items)

{'Cheese', nan, 'Eggs', 'Meat', 'Pencil', 'Milk', 'Wine', 'Diaper', 'Bagel', 'Bread'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [67]:

itemset = set(unique_items)
# encoding the feature
encodedValue = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for i in uncommons:
        labels[i] = 0
    for j in commons:
        labels[j] = 1
    encodedValue.append(labels)

print(labels)

{'Pencil': 0, 'Milk': 0, 'Cheese': 0, 'Diaper': 0, nan: 1, 'Eggs': 1, 'Meat': 1, 'Wine': 1, 'Bagel': 1, 'Bread': 1}


In [68]:
  # create new dataframe from the encoded features
encodeddf = pd.DataFrame(encodedValue)
  # show the new dataframe
encodeddf.head()

,Milk,Bagel,NaN,Cheese,Eggs,Meat,Pencil,Wine,Diaper,Bread
0,0,0,0,1,1,1,1,1,1,1
1,1,0,0,1,0,1,1,1,1,1
2,1,0,1,1,1,1,0,1,0,0
3,1,0,1,1,1,1,0,1,0,0
4,0,0,1,0,0,1,1,1,0,0


In [71]:
# Since, the encoded dataframe consist of the empty column. We will drop the NaN column or u can use the index.
encodeddf = encodeddf.drop(encodeddf.columns[2], axis=1)
encodeddf

,Milk,Bagel,Cheese,Eggs,Meat,Pencil,Wine,Diaper,Bread
0,0,0,1,1,1,1,1,1,1
1,1,0,1,0,1,1,1,1,1
2,1,0,1,1,1,0,1,0,0
3,1,0,1,1,1,0,1,0,0
4,0,0,0,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...
310,0,0,1,1,0,0,0,0,1
311,1,0,0,0,1,1,0,0,0
312,0,0,1,1,1,1,1,1,1
313,0,0,1,0,1,0,0,0,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products.
For this case study, we will min_support=0.2

In [72]:
#Set threshold value untuk digunakan dalam penghitungan support
from mlxtend.frequent_patterns import apriori, association_rules
freqpurchase = apriori(encodeddf, min_support=0.2, use_colnames=True)
freqpurchase.head(33)


c:\Users\yulia\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.501587,(Milk)
1,0.425397,(Bagel)
2,0.501587,(Cheese)
3,0.438095,(Eggs)
4,0.476190,(Meat)
5,0.361905,(Pencil)
6,0.438095,(Wine)
7,0.406349,(Diaper)
8,0.504762,(Bread)
9,0.225397,"(Milk, Bagel)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [73]:
assRules = association_rules(freqpurchase, metric="confidence", min_threshold=0.6)
assRules.head(14)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148,0.350053
1,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148,0.350053
2,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650,0.402687
3,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203,0.469167
4,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540,0.500891
5,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891,0.526414
6,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754,0.330409
7,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624,0.387409
8,"(Meat, Milk)",(Cheese),0.244444,0.501587,0.203175,0.831169,1.657077,0.080564,2.952137,0.524816
9,"(Meat, Cheese)",(Milk),0.323810,0.501587,0.203175,0.627451,1.250931,0.040756,1.337845,0.296655


Provide explanation about __antecedent support__, __consequent support__, __support__, __confidence__, __lift__, __leverage__, __conviction__, __conviction__ and the interpretation from the case above (please use text section)

**Antecedent Support**
It is a support of the ANTECEDENT itemset C.Formula: support(C)=proportion of transaction containing C. Therefore, measures the frequency of the occurence of itemset C in the dataset.

**Consequent Support**
The support of the CONSEQUENCT itemset C. The formula is support(C)=proportion of the transaction that contains C. Therefore, the frequency of the occurence of the antecedent itemset C in the dataset. 

**Support**
The support of the combined itemset AuC. The formula is support(A->C)=support(AuC). Therefore, it measure the frequency of occurence of the combined itemset AuC in the dataset.

**Confidence**
The conditional probability of observing the consequent C given the antecedent A. Formula: confidence (A->C)=support(A->C)/support(A). Therefore, it measure how often the rule A->C is true. 

**Lift**
It compares the likelihood of observing A and C together against the likelihood if they were independent. The formula is lift(A->C)=confidence(A->C)/support(C).

**Leverage**
It measures the difference between the observed frequency of A and C appearing together and the frequency expected under independence. Formula: levarage(A->C)=support(A->C)−support(A)×support(C). Therefore, A value of 0 indicates independence.

**Conviction**
It measures how much the consequent C depends on the antecedent A. Formula: conviction(A→C)=(1−support(C))/(1−confidence(A→C)). Therefore, A high conviction value indicates a strong dependency.
